# 介绍

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/renweibo/stable-diffusion-webui-colab/blob/master/chillout_mix_webui_colab.ipynb)

In [ ]:
#@title 基本配置 {display-mode: "form"}

#@markdown Forms support many types of fields.
root_path = '/content'  #@param {type: "string"}
sd_path = '/content/stable-diffusion-webui'  #@param {type: "string"}
base_install_flag = False #@param {type:"boolean"}
ser_start_flag = False #@param {type:"boolean"}
#@markdown ---

# 安装运行

In [ ]:
#@title 公共函数 {display-mode: "form"}
!pip install -q -U sh
!apt -y update -qq
!apt install -qq aria2
import sh
from pathlib import Path
def aria2c(url, path):
    file = Path(url).name
    sh.aria2c(f"--console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {sd_path}/{path} -o {file}".split())
    !ls -lh -d "$sd_path/$path/$file"

In [ ]:
#@title ## 安装基础软件 {display-mode: "form"}
%cd "$root_path"
%env TF_CPP_MIN_LOG_LEVEL=1

if base_install_flag:
    !wget -q http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
    !wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
    !wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
    !wget -q https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
    !apt install -qq libunwind8-dev
    !dpkg -i *.deb
    %env LD_PRELOAD=libtcmalloc.so
    !rm *.deb

    !pip install -q -U torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 torchtext==0.14.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu116
    !pip install -q -U xformers==0.0.16 triton==2.0.0

In [ ]:
#@title 下载Stable Diffusion Web UI {display-mode: "form"}

%cd "$root_path"
!git clone -b v2.1 https://github.com/camenduru/stable-diffusion-webui
!git clone https://huggingface.co/embed/negative "$sd_path"/embeddings/negative
!git clone https://huggingface.co/embed/lora "$sd_path"/models/Lora/positive
aria2c("https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth", "models/ESRGAN")
!wget https://raw.githubusercontent.com/camenduru/stable-diffusion-webui-scripts/main/run_n_times.py -O "$sd_path"/scripts/run_n_times.py
!git clone https://github.com/deforum-art/deforum-for-automatic1111-webui "$sd_path"/extensions/deforum-for-automatic1111-webui
!git clone https://github.com/camenduru/stable-diffusion-webui-images-browser "$sd_path"/extensions/stable-diffusion-webui-images-browser
!git clone https://github.com/camenduru/stable-diffusion-webui-huggingface "$sd_path"/extensions/stable-diffusion-webui-huggingface
!git clone https://github.com/camenduru/sd-civitai-browser "$sd_path"/extensions/sd-civitai-browser
!git clone https://github.com/kohya-ss/sd-webui-additional-networks "$sd_path"/extensions/sd-webui-additional-networks
!git clone https://github.com/Mikubill/sd-webui-controlnet "$sd_path"/extensions/sd-webui-controlnet
!git clone https://github.com/camenduru/openpose-editor "$sd_path"/extensions/openpose-editor
!git clone https://github.com/jexom/sd-webui-depth-lib "$sd_path"/extensions/sd-webui-depth-lib
!git clone https://github.com/hnmr293/posex "$sd_path"/extensions/posex
!git clone https://github.com/camenduru/sd-webui-tunnels "$sd_path"/extensions/sd-webui-tunnels
!git clone https://github.com/etherealxx/batchlinks-webui "$sd_path"/extensions/batchlinks-webui
!git clone https://github.com/camenduru/stable-diffusion-webui-catppuccin "$sd_path"/extensions/stable-diffusion-webui-catppuccin
!git clone https://github.com/KohakuBlueleaf/a1111-sd-webui-locon "$sd_path"/extensions/a1111-sd-webui-locon
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg "$sd_path"/extensions/stable-diffusion-webui-rembg
!git clone https://github.com/ashen-sensored/stable-diffusion-webui-two-shot "$sd_path"/extensions/stable-diffusion-webui-two-shot
!git clone https://github.com/camenduru/sd_webui_stealth_pnginfo "$sd_path"/extensions/sd_webui_stealth_pnginfo
%cd "$sd_path"
!git reset --hard
!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' "$sd_path"/launch.py
!sed -i -e 's/fastapi==0.90.1/fastapi==0.89.1/g' "$sd_path"/requirements_versions.txt
!mkdir -p "$sd_path"/extensions/deforum-for-automatic1111-webui/models
!sed -i -e 's/\"sd_model_checkpoint\"\,/\"sd_model_checkpoint\,sd_vae\,CLIP_stop_at_last_layers\"\,/g' "$sd_path"/modules/shared.py

In [ ]:
#@title 下载公共数据 {display-mode: "form"}
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_canny-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_depth-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_hed-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_mlsd-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_normal-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_openpose-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_scribble-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/control_seg-fp16.safetensors", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/hand_pose_model.pth", "extensions/sd-webui-controlnet/annotator/openpose")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/body_pose_model.pth", "extensions/sd-webui-controlnet/annotator/openpose")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/dpt_hybrid-midas-501f0c75.pt", "extensions/sd-webui-controlnet/annotator/midas")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_large_512_fp32.pth", "extensions/sd-webui-controlnet/annotator/mlsd")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/mlsd_tiny_512_fp32.pth", "extensions/sd-webui-controlnet/annotator/mlsd")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/network-bsds500.pth", "extensions/sd-webui-controlnet/annotator/hed")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/upernet_global_small.pth", "extensions/sd-webui-controlnet/annotator/uniformer")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_style_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_sketch_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_seg_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_openpose_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_keypose_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_depth_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_color_sd14v1.pth", "extensions/sd-webui-controlnet/models")
aria2c("https://huggingface.co/ckpt/ControlNet/resolve/main/t2iadapter_canny_sd14v1.pth", "extensions/sd-webui-controlnet/models")

In [ ]:
#@title 下载主模型 {display-mode: "form"}
aria2c("https://huggingface.co/ckpt/chilloutmix/resolve/main/chilloutmix_NiPrunedFp32Fix.safetensors", "models/Stable-diffusion")


In [ ]:
#@title 运行服务 {display-mode: "form"}
if ser_start_flag:
    %cd "$sd_path"
    !python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple